# keras使用心得

### 问题一：如何在训练时用F1作为metric输出？

keras官方是不提供F1这个metric的，需要自己实现。
实现代码如下：

In [1]:
def f1score(y_true, y_pred):
    num_tp = K.sum(y_true*y_pred)
    num_fn = K.sum(y_true*(1.0-y_pred))
    num_fp = K.sum((1.0-y_true)*y_pred)
    num_tn = K.sum((1.0-y_true)*(1.0-y_pred))
    f1 = 2.0*num_tp/(2.0*num_tp+num_fn+num_fp)
    return f1

其中`y_true`表示真实标号，`y_pred`表示预测标号（假定标号取值为0或1）。
接着，在模型编译的时候指定我们要使用的metric即可：

In [ ]:
model.compile(optimizer=..., loss=..., metrics=[f1score])

### 问题二：如何在训练模型时同时输出多种loss和metric？（比如同时输出accuracy和f1）
注意看函数`model.compile()`中的metrics参数，首先它是个复数形式，其次它是个列表，说到这应该明白了吧:）  
我们只要把要输出的多个metric丢进列表就可以了：

In [ ]:
model.compile(optimizer=..., loss=..., metrics=[f1score, 'acc'])

### 问题三：如何输出神经网络的模型结构？
1.安装pydot 
不要用pip安装，以免安装到最新版。最新版的pydot中删除了find_graphviz()函数，与keras不兼容。 
到这里安装[1.1.0版本](https://github.com/erocarrera/pydot/tree/v1.1.0)

2.安装graphviz并将可执行文件加入环境变量path 
[下载地址](http://www.graphviz.org/Download_windows.php)
    
3.使用keras调用graphviz进行画图

In [ ]:
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')

### 问题四：`merge`和`Merge`的区别是什么？
首先，`merge()`是个函数，而`Merge`是个类。两者的相同点都是合并输出，不同的是`merge()`只能用于合并两或多个`tensor`，且输出也是`tensor`；而`Merge`既可以用于合并两或多个`tensor`，也可以用于合并两或多个`layer`。

### 问题五：如何使用ModelCheckPoint？
ModelCheckPoint（模型检查点）是keras的一个回调函数，用于在每个epoch后保存模型权重到指定路径，其定义如下：
```
keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, mode='auto')
```
参数

* filename：字符串，保存模型的路径

* monitor：需要监视的值

* verbose：信息展示模式，0或1

* save_best_only：当设置为True时，将只保存在验证集上性能最好的模型

* mode：‘auto’，‘min’，‘max’之一，在save_best_only=True时决定性能最佳模型的评判准则，例如，当监测值为val_acc时，模式应为max，当检测值为val_loss时，模式应为min。在auto模式下，评价准则由被监测值的名字自动推断。

* save_weights_only：若设置为True，则只保存模型权重，否则将保存整个模型（包括模型结构，配置信息等）

filepath可以是格式化的字符串，里面的占位符将会被epoch值和传入on_epoch_end的logs关键字所填入

例如，filename若为`weights.{epoch:02d}-{val_loss:.2f}}.hdf5`，则会生成对应epoch和验证集loss的多个文件。
也就是说如果我们想要保存每个epoch上的模型权重的话需要指定文件名为`weights.{epoch:02d}.hdf5`。

我们可以用load_weights函数读取权重，不过注意的是这个函数仅仅是读取权重而已，还需要在读取权重之前要重建模型结构。